<a href="https://colab.research.google.com/github/super-ruilei/DL-Demos/blob/master/AudioCraft_MusicGen_worked_on_oct_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b magnet https://github.com/camenduru/audiocraft
%cd /content/audiocraft
!apt-get install ffmpeg
!sudo apt-get update -y
!sudo apt-get install python3.9
!python --version
!sudo update-alternatives --config python3
# !python -m pip uninstall pip
# !apt-get install python3.9

!pip install -r requirements.txt
!pip uninstall torch torchvision torchaudio
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!pip install xformers==0.0.22.post4 --index-url https://download.pytorch.org/whl/cu118
!pip install gradio==3.39.0   gradio_client==0.3.0

# !pip install -q av julius omegaconf einops num2words flashy torchmetrics sentencepiece gradio
# !pip install -q https://github.com/camenduru/wheels/releases/download/colab/xformers-0.0.20+1dc3d7a.d20240115-cp310-cp310-linux_x86_64.whl
!pip install -q .

# !python -m demos.musicgen_app --share

/content
Cloning into 'audiocraft'...
remote: Enumerating objects: 1258, done.
remote: Counting objects: 100% (546/546), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 1258 (delta 389), reused 356 (delta 356), pack-reused 712 (from 1)
Receiving objects: 100% (1258/1258), 1.73 MiB | 3.82 MiB/s, done.
Resolving deltas: 100% (715/715), done.
/content/audiocraft
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [57.7

Found existing installation: torch 2.1.0
Uninstalling torch-2.1.0:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/nvfuser/*
    /usr/local/lib/python3.10/dist-packages/torch-2.1.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? y
  Successfully uninstalled torch-2.1.0
Found existing installation: torchvision 0.20.0+cu121
Uninstalling torchvision-0.20.0+cu121:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torchvision-0.20.0+cu121.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libcudart.7ec1eba6.so.12
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libnvjpeg.f00ca762.so.12
    /usr/local/lib

In [7]:
!python -V

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [3]:
from audiocraft.models import MusicGen
from audiocraft.models import MultiBandDiffusion

USE_DIFFUSION_DECODER = False
# Using small model, better results would be obtained with `medium` or `large`.
model = MusicGen.get_pretrained('facebook/musicgen-small')
if USE_DIFFUSION_DECODER:
    mbd = MultiBandDiffusion.get_mbd_musicgen()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


state_dict.bin:   0%|          | 0.00/841M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [6]:
!python -V

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

# 新段落

In [8]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=30
)


In [9]:
import math
import torchaudio
import torch
from audiocraft.utils.notebook import display_audio

def get_bip_bip(bip_duration=0.125, frequency=440,
                duration=0.5, sample_rate=32000, device="cuda"):
    """Generates a series of bip bip at the given frequency."""
    t = torch.arange(
        int(duration * sample_rate), device="cuda", dtype=torch.float) / sample_rate
    wav = torch.cos(2 * math.pi * 440 * t)[None]
    tp = (t % (2 * bip_duration)) / (2 * bip_duration)
    envelope = (tp >= 0.5).float()
    return wav * envelope


In [19]:
# Here we use a synthetic signal to prompt both the tonality and the BPM
# of the generated audio.
import time

time_start=time.time()

res = model.generate_continuation(
    get_bip_bip(0.125).expand(1, -1, -1),
    # 12000, ['Jazz jazz and only jazz','Heartful EDM with beautiful synths and chords like swim in the ocean']
    24000,['Create a calm and peaceful music track inspired by nature, perfect for a 21-year-old who loves serene, nature-themed casual games. The music should evoke a feeling of tranquility, with soft melodies, gentle natural sounds like birdsong or flowing water, and a relaxing tempo that complements the atmosphere of a quiet, beautiful outdoor setting.']
            ,
    progress=True)
time_end=time.time()
print('time cost',time_end-time_start,'s')
display_audio(res, 24000)

time cost 51.526756048202515 s


In [21]:
!python3.9 --version

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968